In [24]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import re

executable_path = r"C:\Program Files\Common Files\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(executable_path=executable_path)

# List of locations
locations = ['lonsdale city market', 'mclaughlin road mississauga', 'loblaws city market brewery district edmonton', 'mcarthur avenue vanier']

grocery_list = []

# Loop through each location
for location in locations:
    url = 'https://www.loblaws.ca/store-locator?type=store&icta=pickup-details-modal'
    driver.get(url)

    # Add a delay of 2 seconds before locating the element
    time.sleep(5)
    
    # Find the location search input field
    search_input = driver.find_element(By.ID, 'location-search__search__input')

    # Clear the input field if it already has a value
    search_input.clear()

    # Enter the desired address or location
    search_input.send_keys(location)

    # Press Enter to perform the search
    search_input.send_keys(Keys.ENTER)
    
    # Add a delay of 2 seconds before locating the element
    time.sleep(5)

    select_location_button = driver.find_element(By.CSS_SELECTOR, 'button.location-set-store__button')
    driver.execute_script("arguments[0].click();", select_location_button)
    
    # Add a delay of 2 seconds before locating the element
    time.sleep(5)

    # Wait for the button to be clickable
    wait = WebDriverWait(driver, 10)
    continue_shopping_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.fulfillment-location-confirmation__actions__button')))
    continue_shopping_button.click()


    url = 'https://www.loblaws.ca/food/fruits-vegetables/c/28000?navid=flyout-L2-fruits-vegetables'
    driver.get(url)

    while True:
        # Wait for the groceries to load
        wait = WebDriverWait(driver, 20)
        groceries = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'product-tracking')))
        
        for grocery in groceries:
            try:
                brand = grocery.find_element(By.CLASS_NAME, 'product-name__item--brand').text
            except:
                brand = "Unknown"

            try:
                product = grocery.find_element(By.CLASS_NAME, 'product-name__item--name').text
            except:
                product = "Unknown"

            try:
                price_text = grocery.find_element(By.CLASS_NAME, 'selling-price-list__item__price--now-price__value').text
                price = float(re.search(r'[\d.]+', price_text).group())
            except:
                price = 0.0

            grocery_dict = {'product brand': brand, "product name": product, "price": price, "store name": location.upper()}
            grocery_list.append(grocery_dict)
        
        try:
            # Scroll to the end of the page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for the page to load
            
            # Find the "Load More" button
            load_more_button = driver.find_element(By.CSS_SELECTOR, '.primary-button.primary-button--load-more-button')
            if load_more_button.is_displayed():
                load_more_button.click()
            else:
                break
        except NoSuchElementException:
            break
        except Exception as e:
            print(str(e))
            break

# Convert the grocery_list to a pandas DataFrame
df = pd.DataFrame(grocery_list)



C:\Users\Pradivan\AppData\Local\Temp\ipykernel_20204\2852072523.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=executable_path)


In [ ]:
browser.quit()

In [25]:
print(df.head(10))

  product brand                      product name  price            store name
0       Unknown                    Bananas, Bunch   2.00  LONSDALE CITY MARKET
1       Unknown                  Strawberries 1LB   6.49  LONSDALE CITY MARKET
2       Unknown                  English Cucumber   1.99  LONSDALE CITY MARKET
3       Unknown           Red Seedless Watermelon   8.99  LONSDALE CITY MARKET
4       Unknown                       Green Onion   1.99  LONSDALE CITY MARKET
5       Unknown                      Red Cherries  13.71  LONSDALE CITY MARKET
6       Unknown                         Asparagus   6.49  LONSDALE CITY MARKET
7       Unknown  Tomato On The Vine Red (1 Bunch)   4.68  LONSDALE CITY MARKET
8       Unknown             Romaine Heart, 3 Pack   3.99  LONSDALE CITY MARKET
9       Unknown              Tomato Beefsteak Red   1.47  LONSDALE CITY MARKET


In [31]:
store_mapping = {
    'LONSDALE CITY MARKET': 'Lonsdale, Vancouver',
    'MCLAUGHLIN ROAD MISSISSAUGA': 'Mclaughlin, Toronto',
    'LOBLAWS CITY MARKET BREWERY DISTRICT EDMONTON': 'Brewery Dist, Edmonton',
    'MCARTHUR AVENUE VANIER': 'Mcarthur, Ottawa'
}

# Replace the store names in the DataFrame
df['store name'] = df['store name'].replace(store_mapping)

# Display the modified DataFrame
print(df.head(20))

      product brand                      product name  price  \
0           Unknown                    Bananas, Bunch   2.00   
1           Unknown                  Strawberries 1LB   6.49   
2           Unknown                  English Cucumber   1.99   
3           Unknown           Red Seedless Watermelon   8.99   
4           Unknown                       Green Onion   1.99   
5           Unknown                      Red Cherries  13.71   
6           Unknown                         Asparagus   6.49   
7           Unknown  Tomato On The Vine Red (1 Bunch)   4.68   
8           Unknown             Romaine Heart, 3 Pack   3.99   
9           Unknown              Tomato Beefsteak Red   1.47   
10          Unknown                   Lettuce Iceberg   3.99   
11          Unknown                      Avocados Bag   6.49   
12          Unknown                             Lemon   0.99   
13  Farmer's Market                     Yellow Onions   3.49   
14          Unknown                  Str

In [32]:
print(len(df))

19495


In [33]:
df.to_csv('Loblaws price city comparison.csv', index=False)

In [9]:
import pandas as pd
# Load the CSV file into a DataFrame
df = pd.read_csv('Loblaws price city comparison.csv')

In [10]:
# Remove duplicate rows based on product name and store name
df_unique = df.drop_duplicates(subset=['product name', 'store name'])

# Save the unique DataFrame to a new CSV file
df_unique.to_csv('Unique_Loblaws_price_comparison.csv', index=False)

In [11]:
# Load the CSV file into a DataFrame
df = pd.read_csv('Unique_Loblaws_price_comparison.csv')

# Create a pivot table to merge the data based on product name and store name
df_pivot = df.pivot_table(index='product name', columns='store name', values='price', aggfunc='first')

# Reset the index to make the product name a column
df_pivot = df_pivot.reset_index()

# Save the merged DataFrame to a new CSV file
df_pivot.to_csv('Merged_Loblaws_price_comparison.csv', index=False)

In [12]:
# Load the merged DataFrame
df_pivot = pd.read_csv('Merged_Loblaws_price_comparison.csv')

# Sort the DataFrame by product name in alphabetical order
df_pivot = df_pivot.sort_values(by='product name')

# Reset the index
df_pivot = df_pivot.reset_index(drop=True)

# Add a product ID column
df_pivot['product id'] = range(1, len(df_pivot) + 1)

# Save the updated DataFrame to a new CSV file
df_pivot.to_csv('Sorted_Merged_Loblaws_price_comparison.csv', index=False)

In [13]:
# Load the sorted and merged DataFrame
df_sorted_merged = pd.read_csv('Sorted_Merged_Loblaws_price_comparison.csv')

# Move 'product id' column as the first column
cols = df_sorted_merged.columns.tolist()
cols = ['product id'] + [col for col in cols if col != 'product id']
df_sorted_merged = df_sorted_merged[cols]

# Save the updated DataFrame to a new CSV file
df_sorted_merged.to_csv('Sorted_Merged_Loblaws_price_comparison.csv', index=False)

In [2]:
import numpy as np
import pandas as pd
df_sorted_merged = pd.read_csv('Sorted_Merged_Loblaws_price_comparison.csv')
# Replace empty cells with NaN
df.replace("", np.nan, inplace=True)
# Save the updated DataFrame to a new CSV file
df_sorted_merged.to_csv('Sorted_Merged_Loblaws_price_comparison.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'Sorted_Merged_Loblaws_price_comparison.csv'